In [2]:
def encode(plain_text, private_key):
    assert plain_text in {0, 1} and private_key in {0, 1}
    return plain_text ^ private_key

In [3]:
def decode(cipher_text, private_key):
    assert cipher_text in {0, 1} and private_key in {0, 1}
    return cipher_text ^ private_key

In [4]:
from itertools import product

In [5]:
?product

Init signature: product(self, /, *args, **kwargs)
Docstring:     
product(*iterables, repeat=1) --> product object

Cartesian product of input iterables.  Equivalent to nested for-loops.

For example, product(A, B) returns the same as:  ((x,y) for x in A for y in B).
The leftmost iterators are in the outermost for-loop, so the output tuples
cycle in a manner similar to an odometer (with the rightmost element changing
on every iteration).

To compute the product of an iterable with itself, specify the number
of repetitions with the optional repeat keyword argument. For example,
product(A, repeat=4) means the same as product(A, A, A, A).

product('ab', range(3)) --> ('a',0) ('a',1) ('a',2) ('b',0) ('b',1) ('b',2)
product((0,1), (0,1), (0,1)) --> (0,0,0) (0,0,1) (0,1,0) (0,1,1) (1,0,0) ...
Type:           type
Subclasses:     

In [6]:
for plaintext, private_key in product({0, 1}, repeat=2):
    ciphertext = encode(plaintext, private_key)
    print(f'key: {private_key}, '
          f'plaintext: {plaintext}, '
          f'ciphertext: {ciphertext}, '
          f'decoded: {decode(ciphertext, private_key)}')

key: 0, plaintext: 0, ciphertext: 0, decoded: 0
key: 1, plaintext: 0, ciphertext: 1, decoded: 0
key: 0, plaintext: 1, ciphertext: 1, decoded: 1
key: 1, plaintext: 1, ciphertext: 0, decoded: 1
